In [1]:
import pandas as pd
import numpy as np

In [2]:
from urllib.request import urlopen
from json import loads

all_enrollment = pd.DataFrame(columns=['year', 'fips', 'race', 'sex', 'students_disc_harass_race', 'students_report_harass_race'])
for year in [2011, 2013, 2015, 2017]:
    url = f"https://educationdata.urban.org/api/v1/schools/crdc/harassment-or-bullying/{year}/race/sex/"
    response = urlopen(url)
    data = loads(response.read())
    df = pd.DataFrame(data['results'], 
                      columns=['year', 'fips', 'race', 'sex', 'students_disc_harass_race', 'students_report_harass_race'])
    all_enrollment = pd.concat([all_enrollment, df])
    print(f'{year} added')

2011 added
2013 added
2015 added
2017 added


In [3]:
# 2011, 2013, 2015, 2017
# cols = ['crdc_id', 'year', 'fips', 'ncessch', 'leaid', 'race', 'sex',
#        'disability', 'lep', 'students_disc_harass_dis',
#        'students_disc_harass_race', 'students_disc_harass_sex',
#        'students_report_harass_dis', 'students_report_harass_race',
#        'students_report_harass_sex']

In [4]:
all_enrollment.head()

,year,fips,race,sex,students_disc_harass_race,students_report_harass_race
0,2011,1,1,1,0,0
1,2011,1,2,1,0,0
2,2011,1,3,1,0,0
3,2011,1,4,1,0,0
4,2011,1,5,1,0,0


In [5]:
df = all_enrollment.rename(columns={
    "students_disc_harass_race": "num_disciplined", 
    "students_report_harass_race": "num_harassed"})
df['race'].unique()

array([1, 2, 3, 4, 5, 6, 7, 99], dtype=object)

In [6]:
df = df[df['race'].isin([i for i in range(1, 8)])]
df = df[(df['num_disciplined'] >= 0) & (df['num_harassed'] >= 0)]

In [7]:
races = {1:'White', 2:'Black', 3:'Hispanic', 4:'Asian', 5:'American Indian or Alaska Native', 
         6:'Native Hawaiian or other Pacific Islander', 7:'Two or more races'}

df = df.replace({"race": races})
df

,year,fips,race,sex,num_disciplined,num_harassed
0,2011,1,White,1,0,0
1,2011,1,Black,1,0,0
2,2011,1,Hispanic,1,0,0
3,2011,1,Asian,1,0,0
4,2011,1,American Indian or Alaska Native,1,0,0
...,...,...,...,...,...,...
9994,2017,1,Hispanic,2,0,0
9995,2017,1,Asian,2,0,0
9996,2017,1,American Indian or Alaska Native,2,0,0
9997,2017,1,Native Hawaiian or other Pacific Islander,2,0,0


In [33]:
final_df_h = df.groupby('year')['num_harassed'].sum().reset_index()
final_df_h['type'] = '# Harassed'
final_df_h = final_df_h.rename(columns={'num_harassed': 'count'})
final_df_h

,year,count,type
0,2011,1272,# Harassed
1,2013,730,# Harassed
2,2015,396,# Harassed
3,2017,274,# Harassed


In [34]:
final_df_d = df.groupby('year')['num_disciplined'].sum().reset_index()
final_df_d['type'] = '# Disciplined'
final_df_d = final_df_d.rename(columns={'num_disciplined': 'count'})
final_df_d

,year,count,type
0,2011,884,# Disciplined
1,2013,592,# Disciplined
2,2015,424,# Disciplined
3,2017,244,# Disciplined


In [35]:
final_df = pd.concat([final_df_d, final_df_h]).reset_index()
final_df

,index,year,count,type
0,0,2011,884,# Disciplined
1,1,2013,592,# Disciplined
2,2,2015,424,# Disciplined
3,3,2017,244,# Disciplined
4,0,2011,1272,# Harassed
5,1,2013,730,# Harassed
6,2,2015,396,# Harassed
7,3,2017,274,# Harassed


In [36]:
final_df.to_csv('final_harass_bully.csv', index=False)

In [39]:
final_harass_df = df.groupby(['year', 'race'])['num_harassed'].sum().reset_index()
final_harass_df = final_harass_df.rename(columns={'num_harassed': 'count'})

final_discipline_df = df.groupby(['year', 'race'])['num_disciplined'].sum().reset_index()
final_discipline_df = final_discipline_df.rename(columns={'num_disciplined': 'count'})

In [40]:
final_harass_df.head()

,year,race,count
0,2011,American Indian or Alaska Native,12
1,2011,Asian,12
2,2011,Black,680
3,2011,Hispanic,86
4,2011,Native Hawaiian or other Pacific Islander,0


In [41]:
final_discipline_df.head()

,year,race,count
0,2011,American Indian or Alaska Native,4
1,2011,Asian,16
2,2011,Black,460
3,2011,Hispanic,40
4,2011,Native Hawaiian or other Pacific Islander,0


In [42]:
final_harass_df.to_csv('final_harass_df.csv', index=False)
final_discipline_df.to_csv('final_discipline_df.csv', index=False)